In [1]:
#@title Define if we are on Colab and mount drive { display-mode: "form" }
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
#@title (COLAB ONLY) Code for setting the environment, installation of packages { display-mode: "form" }
%%capture
if IN_COLAB:
  %env PYTHONPATH=
  !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
  !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
  !./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
  !conda install --channel defaults conda python=3.6 --yes -q
  !conda update --channel defaults --all --yes -q
  import sys
  sys.path.insert(0, "/usr/local/lib/python3.6/site-packages")

  # !pip install fastai pydicom kornia fastcore==1.0.9 --upgrade --quiet
  !conda config --add channels conda-forge
  !conda install -c fastai -c pytorch fastai fastcore=1.0.9 pydicom gdcm kornia scikit-image scikit-learn pandas numpy ipykernel --yes -q

In [ ]:
#@title (COLAB ONLY) Clone GitHub repo { display-mode: "form" }

if IN_COLAB:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  import os
  os.environ['GITHUB_AUTH'] = user + ':' + password
  !git clone https://$GITHUB_AUTH@github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

In [ ]:
if IN_COLAB:
  PATH_DRIVER = '/content/gdrive/My Drive/'
  DATA_FOLDER = 'Dataset/'
else:
  PATH_DRIVER = ''
  DATA_FOLDER = 'data/'

In [2]:
import os

from fastai.basics import *
from fastai.medical.imaging import *

In [3]:
PATH_PREFIX = os.path.join(PATH_DRIVER, DATA_FOLDER, '')
raw_folder = PATH_PREFIX + 'DICOMS'
organize_folder = PATH_PREFIX + 'clasificacion_pendiente'
doubt_folder = PATH_PREFIX + 'doubt'
preprocess_folder = PATH_PREFIX + 'preprocess'

correct_check_DICOM_dict = {
    'SeriesDescription': ['RODILLA AP', 'RODILLAS AP'],
    'BodyPartExamined': ['LOWER LIMB', 'KNEE']
}

doubt_check_DICOM_dict = {
    'SeriesDescription': ['RODILLA INTERCONDILEA', 'RODILLA AP CARGA MONOPODAL', 'RODILLAS EN CARGA', 'T Rodilla lat', 'PIERNA AP MESA']
}

In [4]:
from utils import organize_folders, generate_template, rename_patient

groups = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']
subgroup_length = 20
relation_filepath = PATH_PREFIX + 'relation.csv'
new_numeration = True
copy = True

In [ ]:


def open_name_relation_file(filepath, sep=','):
    """ Extract DataFrame from file containing the relation between original and new files """

    # Check if file exists
    if not os.path.exists(filepath):
        df = pd.DataFrame(columns = ['Original', 'New_Path', 'Filename', 'Original_Filename'])
    else:
        df = pd.read_csv(filepath, sep=sep, index_col=0)

    return df


def save_name_relation_file(relation_df, filepath, sep=','):
    """ Save file containing the relation between original and new files """
    relation_df.to_csv(filepath, sep=sep, index=True)


def get_last_id(relation_df, prefix='IMG_'):
    """ Get the ID of the last filename from current relation of files """

    # Extract the maximum ID currently set
    new_id = relation_df['Filename'].str.split(prefix).str[1].astype(int).max()

    # In case of nan then set it to -1
    if new_id is np.nan:
        new_id = -1

    return new_id


def add_new_relation(relation_df, src_path, src_filename, new_filename):
    
    # Check it does not exist a conflictive addition
    # if src_path in relation_df.index and 'Original_Filename' in relation_df.columns and not np.isnan(relation_df.loc[src_path, 'Original_Filename']):
    if src_path in relation_df.index and 'Original_Filename' in relation_df.columns and relation_df['Original_Filename'].notnull()[src_path]:
        if relation_df.loc[src_path, 'Filename'] != new_filename:
            raise ValueError(f'For file "{src_path}"" there is already a relation with "{relation_df.loc[src_path, "Filename"]}" but it is being added for "{new_filename}"')
    else:
        relation_df.loc[src_path, 'Filename'] = new_filename
        relation_df.loc[src_path, 'Original_Filename'] = src_filename

    return relation_df

In [ ]:
relation_df = organize_folders(raw_folder, organize_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, new_numeration=new_numeration, force_extension='.dcm', copy=copy, check_DICOM_dict=correct_check_DICOM_dict, debug=False)

In [5]:
relation_df = organize_folders(raw_folder, doubt_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, new_numeration=new_numeration, force_extension='.dcm', copy=copy, check_DICOM_dict=doubt_check_DICOM_dict, debug=False)

ValueError: For file "data/DICOMS\03010001161180"" there is already a relation with "IMG_39" but it is being added for "IMG_40"

In [11]:
dicom_files = get_files(organize_folder, extensions='.dcm')
rename_patient(dicom_files)

In [12]:
generate_template(organize_folder, groups, subgroup_length, excel=True)